In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

In [6]:
import os
print(os.getcwd())

/Users/King/Documents/ML/GitHub/Commodites


In [10]:
sugar_data = pd.read_csv('psd_sugar.csv')
delhi_weather = pd.read_csv('Delhi_NCR_1990_2022_Safdarjung.csv')
lucknow_weather = pd.read_csv('Lucknow_1990_2022.csv')
mumbai_weather = pd.read_csv('Mumbai_1990_2022_Santacruz.csv')
sugar_price = pd.read_csv('PSUGAISAUSDM.csv')
print(sugar_price.head())

FileNotFoundError: [Errno 2] No such file or directory: 'psd_sugar.csv'

In [8]:
delhi_weather.rename(columns={'time': 'Date'}, inplace=True)
delhi_weather.rename(columns={'tavg': 'Delhi_avg_weather'}, inplace=True)
delhi_weather.rename(columns={'prcp': 'Delhi_prcp'}, inplace=True)

NameError: name 'delhi_weather' is not defined

In [9]:
delhi_weather['Date'] = pd.to_datetime(delhi_weather['Date'], dayfirst=True, errors='coerce')

#clean version of delhi weather
delhi_weather_data = delhi_weather[['Date', 'Delhi_avg_weather', 'Delhi_prcp']]

NameError: name 'delhi_weather' is not defined

In [ ]:
lucknow_weather.rename(columns={'time': 'Date'}, inplace=True)
lucknow_weather.rename(columns={'tavg': 'Lucknow_avg_weather'}, inplace=True)
lucknow_weather.rename(columns={'prcp': 'Lucknow_prcp'}, inplace=True)

In [ ]:

lucknow_weather['Date'] = pd.to_datetime(lucknow_weather['Date'], dayfirst=True, errors='coerce')

#clean version of lucknow weather
lucknow_weather_data = lucknow_weather[['Date', 'Lucknow_avg_weather', 'Lucknow_prcp']]

In [4]:
mumbai_weather.rename(columns={'time': 'Date'}, inplace=True)
mumbai_weather.rename(columns={'tavg': 'Mumbai_avg_weather'}, inplace=True)
mumbai_weather.rename(columns={'prcp': 'Mumbai_prcp'}, inplace=True)

NameError: name 'mumbai_weather' is not defined

In [ ]:
mumbai_weather['Date'] = pd.to_datetime(mumbai_weather['Date'], dayfirst=True, errors='coerce')


#clean version of mumabi weather 
mumbai_weather_data = mumbai_weather[['Date', 'Mumbai_avg_weather', 'Mumbai_prcp']]

In [ ]:
mumbai_lucknow_weather = pd.merge(mumbai_weather_data, lucknow_weather_data, on='Date')
india_weather = pd.merge(mumbai_lucknow_weather, delhi_weather_data, on='Date')

In [5]:
india_weather['Date'] = pd.to_datetime(india_weather['Date'])

india_weather.set_index('Date', inplace=True)

monthly_weather = india_weather.resample('M').mean() 

#focused on the grow season
growing_season_weather = monthly_weather.loc[
    monthly_weather.index.month.isin([4, 5, 6, 7, 8, 9, 10])].resample('Y').mean()



NameError: name 'india_weather' is not defined

In [ ]:
countries= ['Brazil', 'India']

sugar_data.rename(columns={'Calendar_Year': 'Date'}, inplace=True)
sugar_data.rename(columns={'Value': 'Total_Cane_Sugar_Production_1000MT'}, inplace=True)

In [ ]:
sugar_data['Date'] = pd.to_datetime(sugar_data['Date'], format='%Y', errors='coerce')

brl_inr = sugar_data[sugar_data['Country_Name'].isin(countries)]
sugar_brl_inr = brl_inr[brl_inr['Attribute_Description'] == 'Cane Sugar Production']


#Clear version of brazil and India total sugar production from 1960
brazil_india_sugar = sugar_brl_inr[['Date','Country_Name', 'Total_Cane_Sugar_Production_1000MT']]


In [ ]:
brazil_india_sugar = brazil_india_sugar.copy()
brazil_india_sugar.loc[:, 'Date'] = pd.to_datetime(brazil_india_sugar['Date'])
brazil_india_sugar.set_index('Date', inplace=True)


In [4]:

growing_season_weather.index = pd.to_datetime(growing_season_weather.index, format='%Y-%m-%d')
brazil_india_sugar.index = pd.to_datetime(brazil_india_sugar.index, format='%Y-%m-%d')


growing_season_weather_annual = growing_season_weather.resample('Y').mean()
brazil_india_sugar_annual = brazil_india_sugar.resample('Y').mean()


growing_season_weather_annual.index = growing_season_weather_annual.index.strftime('%d/%m/%Y')
brazil_india_sugar_annual.index = brazil_india_sugar_annual.index.strftime('%d/%m/%Y')


weather_and_production = pd.merge(
    growing_season_weather_annual,
    brazil_india_sugar_annual,
    left_index=True,
    right_index=True,
    how='inner'
)



NameError: name 'growing_season_weather' is not defined

In [219]:
sugar_price.rename(columns={'PSUGAISAUSDM': 'US_cents_per_pound_of_sugar'}, inplace=True)

In [227]:
sugar_price['observation_date'] = pd.to_datetime(sugar_price['observation_date'], format='%Y-%m-%d')
sugar_price.set_index('observation_date', inplace=True)
sugar_price_resampled = sugar_price.resample('Y').mean()

x_y_aligned = pd.merge(
    weather_and_production,    # X: Features
    sugar_price_resampled,     # Y: Target
    left_index=True,
    right_index=True,
    how='inner')
    



KeyError: 'observation_date'

In [228]:
x_y_aligned.head(5)

,Mumbai_avg_weather,Mumbai_prcp,Lucknow_avg_weather,Lucknow_prcp,Delhi_avg_weather,Delhi_prcp,Total_Cane_Sugar_Production_1000MT,PSUGAISAUSDM


In [ ]:
X = X_y_aligned.drop(columns=['US_cents_per_pound_of_sugar'])  # Features
y = X_y_aligned['US_cents_per_pound_of_sugar']                 # target 